In [19]:
import pandas as pd

# Load the dataset with specified column names
try:
    df = pd.read_csv('dataset.csv', names=['left_distance', 'right_distance', 'steering_angle', 'class'])
    print(df.head())  # Display the first few rows of the DataFrame
except FileNotFoundError:
    print("Error: 'dataset.csv' not found. Please upload the file to your Colab environment.")
except pd.errors.ParserError:
    print("Error: Could not parse the CSV file. Please check its format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


   left_distance  right_distance  steering_angle        class
0     190.444080      180.623387             0.1  light right
1     192.705621      183.343043             0.1  light right
2     206.345288      198.078467             0.0     straight
3     215.787143      204.363868             0.0     straight
4     215.077687      208.045892             0.0     straight


In [20]:
# Check NaN values
print(df.isnull().sum())

left_distance        0
right_distance    1152
steering_angle       0
class                0
dtype: int64


In [21]:
# Fill NaN values with the mean of each column
for col in ['left_distance', 'right_distance', 'steering_angle']:
    df[col] = df[col].fillna(df[col].mean())
print(df.head())

# Drop steering angle column
df = df.drop('steering_angle', axis=1)
print(df.head())


   left_distance  right_distance  steering_angle        class
0     190.444080      180.623387             0.1  light right
1     192.705621      183.343043             0.1  light right
2     206.345288      198.078467             0.0     straight
3     215.787143      204.363868             0.0     straight
4     215.077687      208.045892             0.0     straight
   left_distance  right_distance        class
0     190.444080      180.623387  light right
1     192.705621      183.343043  light right
2     206.345288      198.078467     straight
3     215.787143      204.363868     straight
4     215.077687      208.045892     straight


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# Prepare data for CNN
X = df[['left_distance', 'right_distance']].values
y = pd.get_dummies(df['class']).values # One-hot encode the classes

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape data for CNN (add a channel dimension as the last dimension)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the CNN model - Adjust kernel size or padding
model = keras.Sequential(
    [
        layers.Conv1D(32, 2, activation="relu", input_shape=(X_train.shape[1], 1), padding='same'), # Adjust kernel_size to 2 or smaller, or use padding='same'
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(y_train.shape[1], activation="softmax"),  # Output layer with softmax
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split = 0.2)


# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4722 - loss: 1.0722 - val_accuracy: 0.7074 - val_loss: 0.7833
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6925 - loss: 0.7537 - val_accuracy: 0.7455 - val_loss: 0.5836
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7702 - loss: 0.5551 - val_accuracy: 0.8499 - val_loss: 0.4201
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8902 - loss: 0.3970 - val_accuracy: 0.9618 - val_loss: 0.3169
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9376 - loss: 0.2980 - val_accuracy: 0.9517 - val_loss: 0.2511
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9330 - loss: 0.2620 - val_accuracy: 0.9440 - val_loss: 0.2156
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9407 - loss: 0.2241 - val_accuracy: 0.9466 - val_loss: 0.1939
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9656 - loss: 0.1893 - val_accuracy: 0.9466 - val_loss: 0.1867
Epo

In [30]:
import cv2
from google.colab import files

uploaded = files.upload()
image_path = list(uploaded.keys())[0]  # Get the filename

img = cv2.imread(image_path)

# Rescale the image to 640x480
resized_img = cv2.resize(img, (640, 480))

# Save the resized image (optional)
cv2.imwrite('resized_image.jpg', resized_img)

Saving oprpe.png to oprpe.png


True

In [31]:
IM_HEIGHT, IM_WIDTH = 480, 640

def detect_curved_lanes(image):
    def region_of_interest(img, vertices):
        mask = np.zeros_like(img)
        cv2.fillPoly(mask, vertices, 255)
        masked = cv2.bitwise_and(img, mask)
        return masked

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    height, width = image.shape[:2]
    roi_vertices = np.array([[(0, height),
                              (width * 0.45, height * 0.6),
                              (width * 0.55, height * 0.6),
                              (width, height)]], dtype=np.int32)

    roi = region_of_interest(edges, roi_vertices)

    lines = cv2.HoughLinesP(roi, rho=1, theta=np.pi/180, threshold=20, minLineLength=20, maxLineGap=300)
    left_lines, right_lines = [], []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope = (y2 - y1) / (x2 - x1) if x2 != x1 else 0
            if slope < 0:
                left_lines.append(line[0])
            else:
                right_lines.append(line[0])

    def fit_poly(lines):
        if not lines:
            return None
        points_x, points_y = [], []
        for line in lines:
            x1, y1, x2, y2 = line
            points_x.extend([x1, x2])
            points_y.extend([y1, y2])
        if len(points_x) >= 2:
            curve = np.polyfit(points_y, points_x, 2)
            if abs(curve[0]) <= 0.02:
                return curve
        return None

    left_curve, right_curve = fit_poly(left_lines), fit_poly(right_lines)
    result = image.copy()
    plot_y = np.linspace(height * 0.6, height, num=20)

    if left_curve is not None:
        plot_x = left_curve[0] * plot_y**2 + left_curve[1] * plot_y + left_curve[2]
        pts = np.array([np.transpose(np.vstack([plot_x, plot_y]))], dtype=np.int32)
        cv2.polylines(result, pts, False, (255, 0, 0), thickness=4)
    if right_curve is not None:
        plot_x = right_curve[0] * plot_y**2 + right_curve[1] * plot_y + right_curve[2]
        pts = np.array([np.transpose(np.vstack([plot_x, plot_y]))], dtype=np.int32)
        cv2.polylines(result, pts, False, (0, 0, 255), thickness=4)

    return result, left_curve, right_curve

def calculate_lane_distances(left_curve, right_curve, height):
    center_x = IM_WIDTH // 2
    y_eval = height - 1

    left_x = (left_curve[0] * y_eval**2 + left_curve[1] * y_eval + left_curve[2]) if left_curve is not None else None
    right_x = (right_curve[0] * y_eval**2 + right_curve[1] * y_eval + right_curve[2]) if right_curve is not None else None

    left_distance = center_x - left_x if left_x is not None else None
    right_distance = right_x - center_x if right_x is not None else None

    return left_distance, right_distance


In [36]:
# Process the resized image
lane_image, left_curve, right_curve = detect_curved_lanes(resized_img)
left_distance, right_distance = calculate_lane_distances(left_curve, right_curve, IM_HEIGHT)

print(f"Left Distance: {left_distance}")
print(f"Right Distance: {right_distance}")

# Put the data to the model
X_test = np.array([[left_distance, right_distance]])
X_test = scaler.transform(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
predictions = model.predict(X_test)
predicted_class = np.argmax(predictions)

# Print the predicted class value
print(f"Predicted Class: {predicted_class}")


Left Distance: None
Right Distance: 234.3979221596819
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted Class: 2
